# Concurrency Cancellation - Task Timeout and Cancellation Utilities

Provides high-level timeout and cancellation utilities built on anyio's CancelScope.

**Core Features:**
- **Timeout Modes**: Duration-based (`*_after`) and deadline-based (`*_at`)
- **Error Handling**: Fail (raise TimeoutError) or move-on (silent cancel)
- **None Safety**: `None` means no timeout but still cancellable by outer scopes
- **Effective Deadline**: Query ambient deadline from nested cancel scopes
- **AnyIO Integration**: Thin wrapper over anyio with consistent API

In [1]:
from lionherd_core.libs.concurrency import (
    create_task_group,
    current_time,
    effective_deadline,
    fail_after,
    fail_at,
    move_on_after,
    move_on_at,
    sleep,
)

## 1. Basic Timeout - fail_after

`fail_after` creates a context that raises `TimeoutError` if execution exceeds the specified duration.

In [2]:
async def slow_task():
    """Simulates a slow operation."""
    await sleep(2.0)
    return "completed"


# Task completes within timeout
async def test_success():
    with fail_after(3.0):  # 3 second timeout
        result = await slow_task()  # Takes 2 seconds
        print(f"✓ Task completed: {result}")


await test_success()

✓ Task completed: completed


In [3]:
# Task exceeds timeout - raises TimeoutError
async def test_timeout():
    try:
        with fail_after(1.0):  # 1 second timeout
            await slow_task()  # Takes 2 seconds
    except TimeoutError:
        print("✓ TimeoutError raised as expected")


await test_timeout()

✓ TimeoutError raised as expected


## 2. Silent Cancellation - move_on_after

`move_on_after` creates a context that silently cancels execution without raising an error.

In [4]:
async def test_silent_cancel():
    result = None
    with move_on_after(1.0) as scope:  # 1 second timeout
        result = await slow_task()  # Takes 2 seconds - will be cancelled
        print("This won't print - cancelled before completion")

    print("✓ Execution continued silently")
    print(f"  Result: {result}")
    print(f"  Cancelled: {scope.cancel_called}")


await test_silent_cancel()

✓ Execution continued silently
  Result: None
  Cancelled: True


In [5]:
# Use cancelled flag to handle timeout
async def test_handle_cancel():
    with move_on_after(1.0) as scope:
        await slow_task()

    if scope.cancel_called:
        print("✓ Task timed out - using fallback")
        result = "fallback_value"
    else:
        result = "normal_result"

    print(f"  Result: {result}")


await test_handle_cancel()

✓ Task timed out - using fallback
  Result: fallback_value


## 3. Deadline-Based Timeouts - fail_at and move_on_at

Instead of duration, specify an absolute deadline (Unix timestamp).

In [6]:
async def test_deadline():
    now = current_time()
    deadline = now + 1.5  # 1.5 seconds from now

    print(f"Current time: {now:.2f}")
    print(f"Deadline: {deadline:.2f}")

    try:
        with fail_at(deadline):
            await slow_task()  # Takes 2 seconds
    except TimeoutError:
        elapsed = current_time() - now
        print(f"✓ TimeoutError at deadline (elapsed: {elapsed:.2f}s)")


await test_deadline()

Current time: 1015943.12
Deadline: 1015944.62
✓ TimeoutError at deadline (elapsed: 1.50s)


In [7]:
# move_on_at - silent cancellation at deadline
async def test_deadline_silent():
    now = current_time()
    deadline = now + 1.5

    with move_on_at(deadline) as scope:
        await slow_task()

    elapsed = current_time() - now
    print("✓ Silently cancelled at deadline")
    print(f"  Elapsed: {elapsed:.2f}s")
    print(f"  Cancelled: {scope.cancel_called}")


await test_deadline_silent()

✓ Silently cancelled at deadline
  Elapsed: 1.50s
  Cancelled: True


## 4. None Timeouts - Cancellable Without Timeout

Passing `None` creates a cancel scope with no timeout but still cancellable by outer scopes.

In [8]:
async def test_none_timeout():
    # Inner scope: no timeout
    # Outer scope: has timeout
    try:
        with fail_after(1.0):  # Outer timeout  # noqa: SIM117
            with fail_after(None):  # Inner - no timeout, but cancellable
                await slow_task()  # Takes 2 seconds
    except TimeoutError:
        print("✓ Cancelled by outer scope despite inner scope having no timeout")


await test_none_timeout()

✓ Cancelled by outer scope despite inner scope having no timeout


## 5. Nested Cancel Scopes

Cancel scopes can be nested. The most restrictive timeout wins.

In [9]:
async def test_nested_scopes():
    start = current_time()

    try:
        with fail_after(3.0):  # Outer: 3 seconds  # noqa: SIM117
            with fail_after(1.0):  # Inner: 1 second (more restrictive)
                await slow_task()  # Takes 2 seconds
    except TimeoutError:
        elapsed = current_time() - start
        print("✓ Timed out at inner scope (more restrictive)")
        print(f"  Elapsed: {elapsed:.2f}s (expected ~1s, not 3s)")


await test_nested_scopes()

✓ Timed out at inner scope (more restrictive)
  Elapsed: 1.00s (expected ~1s, not 3s)


In [10]:
async def test_nested_mixed():
    """Mix fail_after (raises) and move_on_after (silent)."""
    result = None
    start = current_time()

    with move_on_after(2.0) as outer:  # Outer: silent cancel at 2s
        try:
            with fail_after(1.0):  # Inner: raises at 1s
                await slow_task()  # Takes 2s
        except TimeoutError:
            elapsed = current_time() - start
            print(f"✓ Inner scope raised TimeoutError at {elapsed:.2f}s")
            result = "handled_timeout"

    print(f"  Outer scope cancelled: {outer.cancel_called}")
    print(f"  Final result: {result}")


await test_nested_mixed()

✓ Inner scope raised TimeoutError at 1.00s
  Outer scope cancelled: False
  Final result: handled_timeout


## 6. Effective Deadline - Query Ambient Timeout

`effective_deadline()` returns the absolute deadline from the current nested cancel scopes.

In [11]:
async def test_effective_deadline():
    # No deadline
    deadline = effective_deadline()
    print(f"No cancel scope: {deadline}")

    # Single scope
    with fail_after(5.0):
        deadline = effective_deadline()
        now = current_time()
        remaining = deadline - now if deadline else None
        print(f"Single scope (5s): {remaining:.2f}s remaining")

    # Nested scopes - most restrictive
    with fail_after(10.0), fail_after(2.0):  # More restrictive
        deadline = effective_deadline()
        now = current_time()
        remaining = deadline - now if deadline else None
        print(f"Nested scopes (10s outer, 2s inner): {remaining:.2f}s remaining")


await test_effective_deadline()

No cancel scope: None
Single scope (5s): 5.00s remaining
Nested scopes (10s outer, 2s inner): 2.00s remaining


## 7. Manual Cancellation via CancelScope

`CancelScope` can be cancelled manually, not just by timeout.

In [12]:
async def test_manual_cancel():
    """Demonstrates manual cancellation of tasks via scope.cancel()."""
    with move_on_after(10.0) as scope:  # Long timeout
        async with create_task_group() as tg:
            # Start task - pass callable, NOT coroutine
            # ❌ WRONG: tg.start_soon(slow_task())  # This passes coroutine
            # ✅ CORRECT: tg.start_soon(slow_task)  # This passes callable
            tg.start_soon(slow_task)

            # Wait a bit
            await sleep(0.5)

            # Cancel manually (not via timeout)
            scope.cancel()

            # When scope is cancelled, task group is automatically cancelled
            # No need to manually await - task group context handles cleanup

    # After exiting move_on_after context
    print("✓ Task cancelled manually via scope.cancel()")
    print(f"  scope.cancel_called: {scope.cancel_called}")


await test_manual_cancel()

✓ Task cancelled manually via scope.cancel()
  scope.cancel_called: True


## 8. Real-World Pattern - Retry with Timeout

Combine timeouts with retry logic for robust error handling.

In [13]:
async def unreliable_task(fail_count: int = 2):
    """Task that fails first N times."""
    if not hasattr(unreliable_task, "attempts"):
        unreliable_task.attempts = 0

    unreliable_task.attempts += 1
    await sleep(0.5)

    if unreliable_task.attempts <= fail_count:
        raise RuntimeError(f"Attempt {unreliable_task.attempts} failed")

    return f"Success on attempt {unreliable_task.attempts}"


async def retry_with_timeout(task, max_attempts=3, timeout=5.0):
    """Retry task with overall timeout."""
    attempt = 0
    last_error = None

    try:
        with fail_after(timeout):  # Overall timeout
            while attempt < max_attempts:
                attempt += 1
                try:
                    result = await task()
                    print(f"✓ {result}")
                    return result
                except RuntimeError as e:
                    last_error = e
                    print(f"  Attempt {attempt} failed: {e}")
                    if attempt < max_attempts:
                        await sleep(0.2)  # Backoff

            raise last_error
    except TimeoutError:
        print(f"✗ Retry exceeded timeout ({timeout}s)")
        raise


# Reset attempt counter
unreliable_task.attempts = 0
await retry_with_timeout(unreliable_task, max_attempts=5, timeout=10.0)

  Attempt 1 failed: Attempt 1 failed
  Attempt 2 failed: Attempt 2 failed
✓ Success on attempt 3


'Success on attempt 3'

## 9. Comparison - fail_* vs move_on_*

When to use each timeout mode.

In [14]:
async def compare_modes():
    print("=== fail_after - Raises TimeoutError ===")
    try:
        with fail_after(1.0):
            await slow_task()
        print("Task completed")
    except TimeoutError:
        print("✓ TimeoutError raised - caller knows timeout occurred")

    print("\n=== move_on_after - Silent Cancellation ===")
    with move_on_after(1.0) as scope:
        await slow_task()
        print("This won't print")

    print("✓ Execution continued - must check scope.cancel_called")
    print(f"  scope.cancel_called: {scope.cancel_called}")


await compare_modes()

=== fail_after - Raises TimeoutError ===
✓ TimeoutError raised - caller knows timeout occurred

=== move_on_after - Silent Cancellation ===
✓ Execution continued - must check scope.cancel_called
  scope.cancel_called: True


**Use `fail_after`/`fail_at` when:**
- Timeout is an error condition
- Caller needs to know operation failed
- Critical operations (auth, payments)

**Use `move_on_after`/`move_on_at` when:**
- Timeout is acceptable/expected
- Want graceful degradation
- Optional operations (caching, metrics)
- Need to continue regardless of result

## Summary Checklist

**Concurrency Cancellation Essentials:**
- ✅ `fail_after(seconds)` - Timeout that raises TimeoutError
- ✅ `move_on_after(seconds)` - Timeout that silently cancels
- ✅ `fail_at(deadline)` - Absolute deadline that raises
- ✅ `move_on_at(deadline)` - Absolute deadline that silently cancels
- ✅ `None` timeout - No timeout but still cancellable by outer scopes
- ✅ `effective_deadline()` - Query ambient deadline from nested scopes
- ✅ `CancelScope` - Manual cancellation and status checking
- ✅ Nested scopes - Most restrictive timeout wins

**Design Patterns:**
- Duration vs deadline - Use `*_after` for relative time, `*_at` for absolute
- Fail vs move-on - Raise errors for critical ops, silent cancel for optional
- None safety - `None` means no timeout but preserves cancellability
- Scope nesting - Combine timeouts at different levels

**Next Steps:**
- See `_utils.py` for `current_time()` helper
- See `anyio` docs for advanced CancelScope features
- Combine with `execute()` for concurrent task execution with timeouts